# TASK 2

### For the data detected as abnormal in the task 1, determine if it is an anomaly due to bubble contamination (1), solenoid valve fault (2), or unknown fault (3).

In [4]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.svm import OneClassSVM
from sklearn.metrics import accuracy_score

In [2]:
from google.colab import drive
drive.mount('/content/drive')
PATH = "/content/drive/MyDrive/Progetto Manutenzione"

Mounted at /content/drive


In [2]:
PATH = "./"

In [3]:
# Caricare e concatenare i file CSV (Case001.csv-Case177.csv)
data_frames = []
for i in range(106, 178):
    file_path = (PATH + f'/dataset/train/data/Case{i:03d}.csv')
    df = pd.read_csv(file_path)
    df['Case'] = i
    data_frames.append(df)

data = pd.concat(data_frames, ignore_index=True)
data

,TIME,P1,P2,P3,P4,P5,P6,P7,Case
0,0.000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,106
1,0.001,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,106
2,0.002,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,106
3,0.003,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,106
4,0.004,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,106
...,...,...,...,...,...,...,...,...,...
86467,1.196,1.948734,1.912482,1.904362,1.835654,1.924899,1.857220,1.920020,177
86468,1.197,1.948823,1.899824,1.884360,1.825497,1.927513,1.846068,1.919081,177
86469,1.198,1.957784,1.911383,1.893740,1.859805,1.940953,1.861668,1.950562,177
86470,1.199,1.970451,1.950009,1.945417,1.913911,1.953648,1.835381,1.983321,177


In [6]:
column_names = ['Case', 'Spacecraft', 'Condition', 'SV1', 'SV2', 'SV3', 'SV4', 'BP1', 'BP2', 'BP3', 'BP4', 'BP5', 'BP6', 'BP7', 'BV1']

labels = pd.read_csv(PATH + '/dataset/train/labels.csv', names=column_names)

# Sostituisco "Normal" con 0, "Anomaly" e "Fault" con 1
labels['Condition'] = labels['Condition'].map({'Normal': 0, 'Anomaly': 1, 'Fault': 2})

# Sostituisco "No"/"Yes" con 0/1 nelle colonne da "BP1" a "BV1"
for col in ['BP1', 'BP2', 'BP3', 'BP4', 'BP5', 'BP6', 'BP7', 'BV1']:
    labels[col] = labels[col].map({'No': 0, 'Yes': 1})

labels

,Case,Spacecraft,Condition,SV1,SV2,SV3,SV4,BP1,BP2,BP3,BP4,BP5,BP6,BP7,BV1
0,1,1,0,100,100,100,100,0,0,0,0,0,0,0,0
1,2,1,0,100,100,100,100,0,0,0,0,0,0,0,0
2,3,1,0,100,100,100,100,0,0,0,0,0,0,0,0
3,4,1,0,100,100,100,100,0,0,0,0,0,0,0,0
4,5,1,0,100,100,100,100,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,173,3,1,100,100,100,100,0,0,0,1,0,0,0,0
173,174,3,1,100,100,100,100,0,0,0,0,1,0,0,0
174,175,3,1,100,100,100,100,0,0,0,0,0,1,0,0
175,176,3,1,100,100,100,100,0,0,0,0,0,0,1,0


In [8]:
labels = labels[labels['Condition'] != 0]
labels

,Case,Spacecraft,Condition,SV1,SV2,SV3,SV4,BP1,BP2,BP3,BP4,BP5,BP6,BP7,BV1
105,106,1,2,0,100,100,100,0,0,0,0,0,0,0,0
106,107,1,2,25,100,100,100,0,0,0,0,0,0,0,0
107,108,1,2,50,100,100,100,0,0,0,0,0,0,0,0
108,109,1,2,75,100,100,100,0,0,0,0,0,0,0,0
109,110,1,2,100,0,100,100,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,173,3,1,100,100,100,100,0,0,0,1,0,0,0,0
173,174,3,1,100,100,100,100,0,0,0,0,1,0,0,0
174,175,3,1,100,100,100,100,0,0,0,0,0,1,0,0
175,176,3,1,100,100,100,100,0,0,0,0,0,0,1,0


In [9]:
labels["Condition"].unique()

array([2, 1])

In [13]:
#Creazione dati di train

merged_data = pd.merge(data, labels, on="Case", how="left")

train_data = merged_data[["Case", "Condition", "P1", "P2", "P3", "P4", "P5", "P6", "P7"]]

#train_data = pd.concat([train_data]*2, ignore_index=True)
train_data

,Case,Condition,P1,P2,P3,P4,P5,P6,P7
0,106,2,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
1,106,2,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
2,106,2,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
3,106,2,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
4,106,2,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
...,...,...,...,...,...,...,...,...,...
86467,177,1,1.948734,1.912482,1.904362,1.835654,1.924899,1.857220,1.920020
86468,177,1,1.948823,1.899824,1.884360,1.825497,1.927513,1.846068,1.919081
86469,177,1,1.957784,1.911383,1.893740,1.859805,1.940953,1.861668,1.950562
86470,177,1,1.970451,1.950009,1.945417,1.913911,1.953648,1.835381,1.983321


**TRAIN modello OCSVM**

Viene allenato un modello di One-Class SVM sui dati di tipo ABNORMAL:

- Bubble Anomaly = 1
- SV Fault = 2

entrambe sotto la stessa etichetta, ovvero 0.

Successivamente, il modello allenato viene utilizzato per classificare i dati sprovvisti di etichetta, andando a rilevare le "anomalie", ovvero quei dati che il modello considera diversi da 1 e 2. Questi dati vengono etichettati come -1.

Quindi, in sostanza, avremo:

- casi Bubble Anomaly + SV Fault = 1 (Sono i casi OK)
- casi Unknown Fault = -1 (Sono i casi anomali)

In [15]:
features = ['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7']

X = train_data[features]
y = np.zeros(86472)

In [16]:
X

,P1,P2,P3,P4,P5,P6,P7
0,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
1,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
2,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
3,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
4,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
...,...,...,...,...,...,...,...
86467,1.948734,1.912482,1.904362,1.835654,1.924899,1.857220,1.920020
86468,1.948823,1.899824,1.884360,1.825497,1.927513,1.846068,1.919081
86469,1.957784,1.911383,1.893740,1.859805,1.940953,1.861668,1.950562
86470,1.970451,1.950009,1.945417,1.913911,1.953648,1.835381,1.983321


In [21]:
len(y)

86472

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

X_train, X_test, _, _ = train_test_split(X, y, test_size=0.2, random_state=42)

clf = OneClassSVM(nu=0.1, kernel='rbf', gamma='auto')
clf.fit(X_train)

# Calcoliamo i punteggi di anomalia sui dati di test
y_pred = clf.score_samples(X_test)

# Definiamo una soglia di anomalia (ad esempio, -0.5)
threshold = -0.5
y_pred = (y_pred < threshold).astype(int)

# Calcoliamo le metriche di valutazione
precision = precision_score(y, y_pred)
recall = recall_score(y, y_pred)
f1 = f1_score(y, y_pred)
auroc = roc_auc_score(y, y_pred)

print(f"Precisione: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"Punteggio F1: {f1:.2f}")
print(f"Area sotto la curva ROC: {auroc:.2f}")

In [22]:
model = OneClassSVM()
model.fit(X)

## In locale, il notebook può essere eseguito da qui

In [2]:
#load del modello addestrato su Colab
import pickle
with open('trained_models/task2_OCSVMmodel.pkl', 'rb') as file:
    model = pickle.load(file)
    
print(model)

OneClassSVM()


In [6]:
# Caricare e concatenare i file CSV (Case178.csv-Case223.csv)
data_frames = []
for i in range(178, 223):
    file_path = (PATH + f'/dataset/test/data/Case{i:03d}.csv')
    df = pd.read_csv(file_path)
    df['Case'] = i
    data_frames.append(df)

prediction_data = pd.concat(data_frames, ignore_index=True)
prediction_data

,TIME,P1,P2,P3,P4,P5,P6,P7,Case
0,0.000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,178
1,0.001,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,178
2,0.002,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,178
3,0.003,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,178
4,0.004,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,178
...,...,...,...,...,...,...,...,...,...
54040,1.196,2.999206,2.985047,2.982110,2.970652,3.004048,3.009258,3.006424,222
54041,1.197,2.977775,2.952302,2.945279,2.933262,2.965413,2.964095,2.962946,222
54042,1.198,2.959111,2.930013,2.915038,2.926461,2.932936,2.921135,2.922863,222
54043,1.199,2.948195,2.921907,2.911228,2.918398,2.910006,2.895828,2.893770,222


In [7]:
# Preparazione dei dati di test
features = prediction_data[["P1","P2","P3","P4","P5","P6","P7"]]
case_col = prediction_data[["Case"]]

# Predizione con il modello addestrato
predictions = model.predict(features)
features['Prediction'] = predictions

# Unione dei DataFrame
df_unique = pd.concat([case_col, features], axis=1)

# Calcolo della moda per ogni 'Case'
prediction_result = df_unique.groupby('Case')['Prediction'].apply(lambda x: x.mode().iat[0]).reset_index()

/var/folders/k7/8yszcg2s7nq1zy9gv8rzd3jc0000gn/T/ipykernel_79658/2374725696.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['Prediction'] = predictions


In [8]:
prediction_result

,Case,Prediction
0,178,1
1,179,1
2,180,1
3,181,1
4,182,1
5,183,1
6,184,-1
7,185,1
8,186,1
9,187,1


Da modificare

Unknown = 1
Bubble = 2
SV = 3

## Predizione delle etichette

Soltanto il caso 219 viene classificato come errato, mentre tutti gli altri vengono classificati come:
- Anomaly/Fault -> etichetta = 1
- Unknown Fault -> etichetta = -1

In [ ]:
df1 = pd.read_csv('answer.csv')
df1 = df1[['ID', 'task2']]
df1.columns = ['Case', 'Real']
df1['Real'] = df1['Real'].replace({2: 1, 3: 2})

df2 = prediction_result

merged_df = pd.merge(df2, df1, on='Case', how='left')
merged_df

**CLASSIFICAZIONE BINARIA**

In [30]:
train_data

,Case,Condition,P1,P2,P3,P4,P5,P6,P7
0,106,2,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
1,106,2,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
2,106,2,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
3,106,2,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
4,106,2,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
...,...,...,...,...,...,...,...,...,...
86467,177,1,1.948734,1.912482,1.904362,1.835654,1.924899,1.857220,1.920020
86468,177,1,1.948823,1.899824,1.884360,1.825497,1.927513,1.846068,1.919081
86469,177,1,1.957784,1.911383,1.893740,1.859805,1.940953,1.861668,1.950562
86470,177,1,1.970451,1.950009,1.945417,1.913911,1.953648,1.835381,1.983321


In [31]:
features = ['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7']

X = train_data[features]
y = train_data["Condition"]

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

models = [
    ('Logistic Regression', LogisticRegression()),
    ('Decision Tree', DecisionTreeClassifier()),
    ('Random Forest', RandomForestClassifier()),
    ('AdaBoost', AdaBoostClassifier()),
    ('Gradient Boosting', GradientBoostingClassifier()),
    ('Linear SVM', LinearSVC()),
    ('K-Nearest Neighbors', KNeighborsClassifier()),
    ('Gaussian Naive Bayes', GaussianNB()),
    ('Linear Discriminant Analysis', LinearDiscriminantAnalysis()),
    ('Quadratic Discriminant Analysis', QuadraticDiscriminantAnalysis()),
    ('Multi-layer Perceptron', MLPClassifier())
]

# Addestramento e valutazione dei modelli
for model_name, model in models:

    print(f"Training {model_name}...")
    model.fit(X_train, y_train)

    print(f"Prediction on {model_name}...")
    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    print(f'{model_name}: Accuracy = {accuracy:.2f}')

Training Logistic Regression...
Prediction on Logistic Regression...
Logistic Regression: Accuracy = 0.67
Training Decision Tree...
Prediction on Decision Tree...
Decision Tree: Accuracy = 0.90
Training Random Forest...
Prediction on Random Forest...
Random Forest: Accuracy = 0.95
Training AdaBoost...
Prediction on AdaBoost...
AdaBoost: Accuracy = 0.67
Training Gradient Boosting...
Prediction on Gradient Boosting...
Gradient Boosting: Accuracy = 0.68
Training Linear SVM...


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Prediction on Linear SVM...
Linear SVM: Accuracy = 0.67
Training K-Nearest Neighbors...
Prediction on K-Nearest Neighbors...
K-Nearest Neighbors: Accuracy = 0.78
Training Gaussian Naive Bayes...
Prediction on Gaussian Naive Bayes...
Gaussian Naive Bayes: Accuracy = 0.66
Training Linear Discriminant Analysis...
Prediction on Linear Discriminant Analysis...
Linear Discriminant Analysis: Accuracy = 0.67
Training Quadratic Discriminant Analysis...
Prediction on Quadratic Discriminant Analysis...
Quadratic Discriminant Analysis: Accuracy = 0.65
Training Multi-layer Perceptron...
Prediction on Multi-layer Perceptron...
Multi-layer Perceptron: Accuracy = 0.68


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [5]:
#Creazione dati da classificare

# Caricare e concatenare i file CSV (Case178.csv-Case223.csv)
data_frames = []
for i in range(178, 224):
    file_path = (PATH + f'/dataset/test/data/Case{i:03d}.csv')
    df = pd.read_csv(file_path)
    df['Case'] = i
    data_frames.append(df)

prediction_data = pd.concat(data_frames, ignore_index=True)
prediction_data

,TIME,P1,P2,P3,P4,P5,P6,P7,Case
0,0.000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,178
1,0.001,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,178
2,0.002,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,178
3,0.003,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,178
4,0.004,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,178
...,...,...,...,...,...,...,...,...,...
55241,1.196,2.017816,2.016297,2.012680,2.044149,2.037457,2.033628,2.069056,223
55242,1.197,2.039878,2.055107,2.054626,2.071298,2.072899,2.076782,2.093548,223
55243,1.198,2.054024,2.081678,2.094493,2.096693,2.095300,2.107353,2.138567,223
55244,1.199,2.057180,2.094954,2.097106,2.153193,2.099423,2.112326,2.127471,223


In [6]:
anomaly_to_exclude = [184, 192, 200, 207, 218, 222]
normal_to_exclude = [180, 182, 183, 185, 187, 189, 191, 194, 195, 198, 201, 203, 206, 208, 210, 213, 215, 217, 220, 223]

prediction_data = prediction_data[~prediction_data['Case'].isin(anomaly_to_exclude)]
prediction_data = prediction_data[~prediction_data['Case'].isin(normal_to_exclude)]

prediction_data

,TIME,P1,P2,P3,P4,P5,P6,P7,Case
0,0.000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,178
1,0.001,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,178
2,0.002,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,178
3,0.003,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,178
4,0.004,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,178
...,...,...,...,...,...,...,...,...,...
52839,1.196,2.047358,2.106033,2.116920,2.169856,2.066395,2.086070,2.086723,221
52840,1.197,2.062200,2.104518,2.115623,2.195780,2.089076,2.099911,2.095505,221
52841,1.198,2.062772,2.093498,2.096172,2.163714,2.091489,2.115760,2.108553,221
52842,1.199,2.045088,2.077390,2.083716,2.120518,2.088176,2.111408,2.103875,221


In [7]:
import pickle
with open('trained_models/task2_RFmodel.pkl', 'rb') as file:
    model = pickle.load(file)

In [ ]:
model = models[2][1]

In [8]:
# Preparazione dei dati di test
features = prediction_data[["P1","P2","P3","P4","P5","P6","P7"]]
case_col = prediction_data[["Case"]]

# Predizione con il modello addestrato
predictions = model.predict(features)
features['Prediction'] = predictions

# Unione dei DataFrame
df_unique = pd.concat([case_col, features], axis=1)

# Calcolo della moda per ogni 'Case'
prediction_result = df_unique.groupby('Case')['Prediction'].apply(lambda x: x.mode().iat[0]).reset_index()

/var/folders/k7/8yszcg2s7nq1zy9gv8rzd3jc0000gn/T/ipykernel_92268/2374725696.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['Prediction'] = predictions


In [9]:
prediction_result

,Case,Prediction
0,178,1
1,179,2
2,181,2
3,186,1
4,188,2
5,190,2
6,193,1
7,196,1
8,197,1
9,199,2


In [10]:
df1 = pd.read_csv('answer.csv')
df1 = df1[['ID', 'task2']]
df1.columns = ['Case', 'Real']
df1['Real'] = df1['Real'].replace({2: 1, 3: 2})

df2 = prediction_result

merged_df = pd.merge(df2, df1, on='Case', how='left')
merged_df.to_csv('./result/risultati_task2.csv', index=False)
merged_df

,Case,Prediction,Real
0,178,1,1
1,179,2,2
2,181,2,2
3,186,1,1
4,188,2,2
5,190,2,2
6,193,1,1
7,196,1,1
8,197,1,1
9,199,2,2


In [11]:
# Calcola il numero totale di righe
total_rows = len(merged_df)

# Calcola il numero di righe con predizioni errate
wrong_predictions = merged_df[merged_df['Prediction'] != merged_df['Real']]
num_wrong = len(wrong_predictions)

# Calcola la percentuale di dati sbagliati
percentage_wrong = (num_wrong / total_rows) * 100

print(f"Numero totale di righe: {total_rows}")
print(f"Numero di righe con predizioni errate: {num_wrong}")
print(f"Percentuale di dati sbagliati: {percentage_wrong:.2f}%")

Numero totale di righe: 20
Numero di righe con predizioni errate: 1
Percentuale di dati sbagliati: 5.00%
